# Comparação de Benchmark Logscan

Este notebook compara a precisão do Logscan em dois datasets:
1. **Loghub 2k** (Dataset original, resultados em `Logscan_benchmark_result.csv`)
2. **Loghub 2.0** (Dataset atualizado, resultados em `Logscan-llm_benchmark_result.csv`)

O objetivo é analisar o impacto da atualização do dataset na precisão do parsing.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Caminhos para os arquivos de resultados
result_2k_path = "../Logscan_benchmark_result.csv"
result_2_0_path = "../Logscan-llm_benchmark_result.csv"

# Verificar se os arquivos existem
has_2k = os.path.exists(result_2k_path)
has_2_0 = os.path.exists(result_2_0_path)

if not has_2k and not has_2_0:
    print("Nenhum arquivo de benchmark encontrado. Execute 'python -m logscan' e 'python -m logscan-llm' na raiz para gerá-los.")
else:
    dfs = []
    if has_2k:
        df_2k = pd.read_csv(result_2k_path)
        df_2k = df_2k.rename(columns={'Accuracy': 'Accuracy_2k'})
        if 'Dataset' not in df_2k.columns and df_2k.index.name == 'Dataset':
             df_2k = df_2k.reset_index()
        dfs.append(df_2k.set_index('Dataset'))
    
    if has_2_0:
        df_2_0 = pd.read_csv(result_2_0_path)
        df_2_0 = df_2_0.rename(columns={'Accuracy': 'Accuracy_2.0'})
        if 'Dataset' not in df_2_0.columns and df_2_0.index.name == 'Dataset':
             df_2_0 = df_2_0.reset_index()
        dfs.append(df_2_0.set_index('Dataset'))
    
    # Combinar os dataframes
    if len(dfs) == 2:
        df_compare = pd.concat(dfs, axis=1, join='inner') # 'inner' para comparar apenas datasets comuns
        df_compare['Diferença'] = df_compare['Accuracy_2.0'] - df_compare['Accuracy_2k']
        
        print("Tabela de Comparação:")
        display(df_compare)
        
        # Plotar
        ax = df_compare[['Accuracy_2k', 'Accuracy_2.0']].plot(kind='bar', figsize=(12, 6))
        plt.title('Comparação de Acurácia Logscan: Loghub 2k vs Loghub 2.0')
        plt.ylabel('Acurácia')
        plt.xlabel('Dataset')
        plt.ylim(0, 1.1)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.legend(loc='lower right')
        plt.tight_layout()
        plt.show()
        
        print("\nAnálise:")
        for dataset, row in df_compare.iterrows():
            diff = row['Diferença']
            status = "melhorou" if diff > 0 else "piorou" if diff < 0 else "manteve"
            print(f"- {dataset}: {status} ({diff:+.4f})")
            
    elif has_2k:
        print("Apenas resultados do Loghub 2k encontrados:")
        display(df_2k)
    elif has_2_0:
        print("Apenas resultados do Loghub 2.0 encontrados:")
        display(df_2_0)
